In [ ]:
import geocoder
import ephem
import ephem.stars
import datetime
import math
import cmath
import sympy
import functools
import numpy as np
import qutip
import random
import vpython
import mpmath
import scipy

def xyz_c(x, y, z):
    if z == 1:
        return float('inf') 
    else:
        return complex(x/(1-z), y/(1-z))

def c_xyz(c):
    if c == float('inf'):
        return [0,0,1]
    x = c.real
    y = c.imag
    return [(2*x)/(1.+(x**2)+(y**2)), \
           (2*y)/(1.+(x**2)+(y**2)), \
           (-1.+(x**2)+(y**2))/(1.+(x**2)+(y**2))]

def altaz_xyz(altitude, azimuth):
    x = math.sin(azimuth)*math.cos(altitude)
    y = math.cos(altitude)*math.cos(azimuth)
    z = math.sin(altitude)
    return [x, y, z]

def xyz_spherical(x, y, z):
    r = math.sqrt(x**2 + y**2 + z**2)
    theta = math.acos(z/r)
    phi = math.atan(y/z)
    return [r, theta, phi]

def spherical_xyz(r, theta, phi):
    x = r*math.sin(theta)*math.cos(phi)
    y = r*math.sin(theta)*math.sin(phi)
    z = r*math.cos(theta)
    return [x, y, z]

def spherical_state(r, theta, phi):
    return np.array([math.cos(theta/2), cmath.exp(complex(0,1)*phi)*theta/2])

def xyz_qubit(x, y, z):
    return (x*qutip.sigmax() + y*qutip.sigmay() + z*qutip.sigmaz()).full()

def txyz_qubit(t, x, y, z):
    return (t*qutip.identity(2) + x*qutip.sigmax(2) + y*qutip.sigmay() + z*qutip.sigmaz()).full()

def scalar_product(m, n):
    return 0.5*np.trace(np.dot(np.conjugate(m).T, n))

def qubit_txyz(m):
    t = scalar_product(np.eye(2), m).real
    x = scalar_product(qutip.sigmax().full(), m).real
    y = scalar_product(qutip.sigmay().full(), m).real
    z = scalar_product(qutip.sigmaz().full(), m).real
    return [t/math.sqrt(t**2+x**2+y**2+z**2), x/t, y/t, z/t]

def distinguish_qubits(state):
    state.dims = [[2,2,2],[1,1,1]]
    traces = [state.ptrace(i).full() for i in range(3)]
    state.dims = [[8],[1]]
    return traces

def upgrade(qubit_measurement, q):
    total_measurement = None
    if q == 0:
        total_measurement = qutip.tensor(qubit_measurement, qutip.identity(2), qutip.identity(2))
    elif q == 1:
        total_measurement = qutip.tensor(qutip.identity(2), qubit_measurement, qutip.identity(2))
    elif q == 2:
        total_measurement = qutip.tensor(qutip.identity(2), qutip.identity(2), qubit_measurement)
    return total_measurement

def roots_polynomial(roots):
    s = sympy.symbols("s")
    polynomial = sympy.Poly(functools.reduce(lambda a, b: a*b, [s+root for root in roots]), domain="CC")
    return polynomial.coeffs()

def polynomial_roots(polynomial):
    try:
        roots = [complex(root) for root in mpmath.polyroots(polynomial)]
    except:
        return [complex(0,0) for i in range(len(polynomial)-1)]
    return roots

def combos(a,b):
    f = math.factorial
    return f(a) / f(b) / f(a-b)

def polynomial_state(polynomial):
    coordinates = [polynomial[i]/(((-1)**i) * math.sqrt(combos(len(polynomial)-1,i))) for i in range(len(polynomial))]
    return qutip.Qobj(np.array(coordinates).T)

def state_polynomial(v):
    polynomial = v.T.tolist()
    return [(((-1)**i) * math.sqrt(combos(len(polynomial)-1,i))) * polynomial[i] for i in range(len(polynomial))]

def state_xyz(v):
    return [c_xyz(root) for root in polynomial_roots(state_polynomial(v))]

def state_syms(v):
    return [xyz_qubit(*xyz) for xyz in state_xyz(v)]

def state_spherical(state):
    return [xyz_spherical(*xyz) for xyz in state_xyz(state)]

def state_states(state):
    return [spherical_state(*spherical) for spherical in state_spherical(state)]

def states_state(states):
    return polynomial_state(roots_polynomial([xyz_c(*spherical_xyz(*state_spherical(state)[0])) for state in states]))

def make_unitary(h, dt):
    return scipy.linalg.expm(-2*math.pi*complex(0,1)*h*dt)

def A(t):
    return np.array([[np.cosh(t), np.sinh(t)], [np.sinh(t), np.cosh(t)]])

def N(t):
    return np.array([[1, 0], [t, 1]])

def K(t):
    return np.array([[np.cos(t), -1*np.sin(t)],[np.sin(t), np.cos(t)]])

def apply_mobius(m, states):
    return [np.dot(m, state) for state in states]

def qutip_numpy(state):
    return state.full().T[0]

STAR_NAMES = [star.split(",")[0] for star in ephem.stars.db.split("\n")][:-1]

class Sphere:
    def __init__(self):
        self.latitude, self.longitude = 0, 0
        self.delta = 0.005
        
        self.astronomy_on = False
        self.astronomical_evolution_on = False
        self.time = None
        self.time_delta = datetime.timedelta(minutes=1)
        self.address = "476 Jefferson St, Brooklyn"
        self.n_fixed_stars = len(STAR_NAMES)
        self.fixed_stars = [spherical_state(1, random.uniform(0, math.pi), random.uniform(0, 2*math.pi)) for i in range(self.n_fixed_stars)]
        
        self.state = qutip.rand_ket(8)

        self.hamiltonian_on = False
        self.hamiltonian_sign = 1
        self.hamiltonian = qutip.rand_herm(8)

        self.fixed_pole = (0, "interior")
        
        vpython.scene.width = 1000
        vpython.scene.height = 1000
        vpython.scene.range = 1.5
        vpython.scene.forward = vpython.vector(0, 1, 0)
        vpython.scene.up = vpython.vector(-1, 0, 0)
        
        self.vsphere = vpython.sphere(pos=vpython.vector(0,0,0), radius=1, color=vpython.color.blue, opacity=0.5)
        self.vearth = vpython.sphere(pos=vpython.vector(0,0,0), radius=0.1, color=vpython.color.cyan, opacity=0.5, emissive=True)
        self.vobserver = vpython.sphere(pos=vpython.vector(0,0,0), radius=0.01, color=vpython.color.yellow, opacity=0.5, emissive=True, make_trail=True)

        self.vstamp = vpython.label(pos=vpython.vector(0,0,0), text="", height=10, opacity=0.6, visible=False)
        self.vfixed_stars = [vpython.sphere(radius=0.01, emissive=True, color=vpython.color.white, make_trail=True) for i in range(self.n_fixed_stars)]

        self.vplanets = [vpython.sphere(radius=0.1, emissive=True, make_trail=False) for i in range(7)]
        self.vplanets[0].color = vpython.color.yellow
        self.vplanets[1].color = vpython.color.white
        self.vplanets[2].color = vpython.color.blue
        self.vplanets[3].color = vpython.color.green
        self.vplanets[4].color = vpython.color.red
        self.vplanets[5].color = vpython.color.orange
        self.vplanets[6].color = vpython.color.gray(0.5)
        
        self.vqubits = [vpython.sphere(radius=0.1, emissive=True, make_trail=False) for i in range(3)]
        self.veigs = [vpython.sphere(color=vpython.color.black, radius=0.05, emissive=True, make_trail=False) for i in range(6)]
        self.vlines = [vpython.curve(pos=[vpython.vector(0,0,0), vpython.vector(0,0,0)]) for i in range(3)]
        self.vlines[0].color = vpython.color.red
        self.vlines[1].color = vpython.color.green
        self.vlines[2].color = vpython.color.blue
        
        self.touched = True
        self.virgin = True

    def fix(self, planets_XYZ):
        ordering = []
        for i in range(7):
            vplanet = self.vplanets[i]
            x, y, z = vplanet.pos.x, vplanet.pos.y, vplanet.pos.z
            current_xyz = np.array([x, y, z])
            lowest_distance = None
            winners = []
            for j in range(7):
                new_xyz = np.array(planets_XYZ[j])
                distance = np.linalg.norm(current_xyz-new_xyz)
                if lowest_distance == None or distance < lowest_distance:
                    lowest_distance = distance
                    winners.append(j)
            for m in range(len(winners)-1, -1, -1):
                if winners[m] not in ordering:
                    ordering.append(winners[m])
                    break
                if m == 0:
                    for r in range(i, 7):
                        if r not in ordering:
                            ordering.append(i)
                            break
                        if r == 6:
                            for p in range(0, i):
                                if p not in ordering:
                                    ordering.append(p)
                                    break
        return [planets_XYZ[i] for i in ordering]
    
    def astronomical_update(self):
        observer = ephem.Observer()
        observer.date = self.time
        observer.lat = self.latitude
        observer.lon = self.longitude
        ephem_stars = [ephem.star(star_name, observer) for star_name in STAR_NAMES]
        ephem_planets = [ephem.Sun(observer), ephem.Moon(observer), ephem.Mercury(observer),\
                ephem.Venus(observer), ephem.Mars(observer), ephem.Jupiter(observer),\
                ephem.Saturn(observer)]
        fixed_XYZ = [altaz_xyz(fixed_star.alt, fixed_star.az) for fixed_star in ephem_stars]
        planets_XYZ = [altaz_xyz(planet.alt, planet.az) for planet in ephem_planets]
        return fixed_XYZ, planets_XYZ
    
    def revolve(self):
        if self.astronomy_on:
            if self.astronomical_evolution_on:
                fixed_XYZ, planets_XYZ = self.astronomical_update()
                self.fixed_stars = [spherical_state(*xyz_spherical(*xyz)) for xyz in fixed_XYZ]
                self.state = polynomial_state(roots_polynomial([xyz_c(*xyz) for xyz in planets_XYZ]))
                self.time += self.time_delta
                self.touched = True
        if self.hamiltonian_on:
            unitary = qutip.Qobj(make_unitary(self.hamiltonian.full(), self.delta))
            if self.hamiltonian_sign == -1:
                unitary = unitary.dag()
            self.state = unitary*self.state
            self.touched = True
        if self.touched:
            planets_XYZ = state_xyz(qutip_numpy(self.state))
            if not self.virgin:
                planets_XYZ = self.fix(planets_XYZ) 
            if self.virgin:
                self.virgin = False
            distinguishable_qubits = distinguish_qubits(self.state)
            qubits_TXYZ = [qubit_txyz(qubit) for qubit in distinguishable_qubits]
            eigs_XYZ = []
            for q in distinguishable_qubits:
                eigenvalues, eigenvectors = np.linalg.eig(q)
                for v in eigenvectors:
                    eigs_XYZ.append(state_xyz(np.conjugate(v))[0])
            fixed_XYZ = [state_xyz(fixed_star)[0] for fixed_star in self.fixed_stars]
            self.draw(planets_XYZ, qubits_TXYZ, eigs_XYZ, fixed_XYZ)
            touched = False

    def draw(self, planets_XYZ, qubits_TXYZ, eigs_XYZ, fixed_XYZ):
        vpython.rate(300)
        if self.astronomy_on:
            self.vstamp.text = self.time.strftime("%c")+"\nlat: %.2f lng: %.2f" % (self.latitude, self.longitude)
        self.vobserver.pos = vpython.vector(*[0.1*coord for coord in spherical_xyz(1, self.latitude, self.longitude)])
        for i in range(len(self.vplanets)):
            self.vplanets[i].pos = vpython.vector(*planets_XYZ[i])
        for i in range(len(self.vqubits)):
            self.vqubits[i].color = vpython.color.hsv_to_rgb(vpython.vector(float(qubits_TXYZ[i][0]),1,1))
            self.vqubits[i].pos = vpython.vector(*qubits_TXYZ[i][1:])
        for i in range(len(self.veigs)):
            self.veigs[i].pos = vpython.vector(*eigs_XYZ[i])
        for i in range(len(self.vlines)):
            self.vlines[i].modify(0, pos=vpython.vector(*eigs_XYZ[2*i]))
            self.vlines[i].modify(1, pos=vpython.vector(*eigs_XYZ[2*i+1]))
        for i in range(len(self.vfixed_stars)):
            self.vfixed_stars[i].pos = vpython.vector(*fixed_XYZ[i])
    
    def set_pole(self, i, kind):
        self.fixed_pole = i, kind

    def collapse(self, q, sign):
        total_measurement = upgrade(qutip.Qobj(distinguish_qubits(self.state)[q]), q)        
        eigenvalues, eigenvectors = total_measurement.eigenstates()
        projection = None
        if eigenvalues[0] > eigenvalues[4]:
            sign *= -1
        if sign == -1:
            projection = sum([eigenvectors[i] for i in range(4)])
        elif sign == 1:
            projection = sum([eigenvectors[i] for i in range(4, 8)])
        self.state = projection
        self.state.dims = [[8],[1]]
        self.refresh()
        self.touched = True

    def sphere_step(self, kind, sign):
        if kind == "latitude":
            self.latitude += sign*self.delta
        elif kind == "longitude":
            self.longitude += sign*self.delta
        elif kind == "step":
            self.delta += sign*0.001
            self.time_delta = datetime.timedelta(seconds=(self.time_delta.seconds+sign*60))
        self.touched = True

    def mobius_step(self, m, sign):
        M = None
        if m == "A":
            M = A(sign*0.01)
        elif m == "N":
            M = N(sign*0.01)
        elif m == "K":
            M = K(sign*0.01)
        self.fixed_stars = apply_mobius(M, self.fixed_stars)
        planets_XYZ = self.fix(state_xyz(qutip_numpy(self.state))) 
        states = [spherical_state(*xyz_spherical(*xyz)) for xyz in planets_XYZ]
        new_states = apply_mobius(M, states)
        self.state = states_state(new_states)
        self.touched = True

    def qubit_step(self, q, sign):
        if self.fixed_pole == None:
            return
        pole, kind = self.fixed_pole
        operator = None
        if kind == "surface":
            operator = state_syms(qutip_numpy(self.state))[pole]
        elif kind == "interior":
            operator = distinguish_qubits(self.state)[pole]
        operator = qutip.Qobj(make_unitary(operator, self.delta))
        if sign == -1:
            operator = operator.dag()
        full_operator = upgrade(operator, q)
        full_operator.dims = [[8],[8]]
        self.state = full_operator*self.state
        self.refresh()
        self.touched = True

    def toggle_hamiltonian(self):
        if self.hamiltonian_on:
            self.hamiltonian_on = False
        else:
            self.hamiltonian_on = True

    def toggle_hamiltonian_sign(self):
        if self.hamiltonian_sign == 1:
            self.hamiltonian_sign = -1
        elif self.hamiltonian_sign == -1:
            self.hamiltonian_sign = 1

    def random_state(self):
        self.state = qutip.rand_ket(8)
        self.fixed_stars = [spherical_state(1, random.uniform(0, math.pi), random.uniform(0, 2*math.pi)) for i in range(self.n_fixed_stars)]
        self.touched = True
        self.virgin = True
        self.revolve()
        self.refresh()

    def random_hamiltonian(self):
        self.hamiltonian = qutip.rand_herm(8)

    def reorient(self):
        vpython.scene.forward = vpython.vector(0, 1, 0)
        vpython.scene.up = vpython.vector(-1, 0, 0)
        self.touched = True

    def refresh(self):
        self.vobserver.clear_trail()
        for vplanet in self.vplanets:
            vplanet.clear_trail()
        for vqubit in self.vqubits:
            vqubit.clear_trail()
        for vfixed_star in self.vfixed_stars:
            vfixed_star.clear_trail()
            
    def load_astronomy(self):
        try:
            self.latitude, self.longitude = geocoder.google(self.address).latlng
        except:
            self.latitude, self.longitude = 0, 0
        self.time = datetime.datetime.now()
        self.vfixed_stars[STAR_NAMES.index("Sirius")].color = vpython.color.red
        self.vfixed_stars[STAR_NAMES.index("Betelgeuse")].color = vpython.color.yellow
        self.vfixed_stars[STAR_NAMES.index("Rigel")].color = vpython.color.yellow
        self.vfixed_stars[STAR_NAMES.index("Bellatrix")].color = vpython.color.yellow
        self.vfixed_stars[STAR_NAMES.index("Mintaka")].color = vpython.color.yellow
        self.vfixed_stars[STAR_NAMES.index("Alnilam")].color = vpython.color.yellow
        self.vfixed_stars[STAR_NAMES.index("Alnitak")].color = vpython.color.yellow
        self.vfixed_stars[STAR_NAMES.index("Saiph")].color = vpython.color.yellow
        self.vfixed_stars[STAR_NAMES.index("Polaris")].color = vpython.color.blue
    
    def unload_astronomy(self):
        self.fixed_stars = [spherical_state(1, random.uniform(0, math.pi), random.uniform(0, 2*math.pi)) for i in range(self.n_fixed_stars)]
        for vfixed_star in self.vfixed_stars:
            vfixed_star.color = vpython.color.white
            
    def toggle_astronomy(self):
        if self.astronomy_on:
            self.astronomy_on = False
            self.astronomical_evolution_on = False
            self.unload_astronomy()
            self.vstamp.visible = False
        else:
            self.astronomy_on = True
            self.load_astronomy()
            self.astronomical_evolution_on = True
            self.vstamp.visible = True
        self.revolve()    
        self.refresh()
    
    def toggle_astronomical_time(self):
        if self.astronomical_evolution_on:
            self.astronomical_evolution_on = False
        else:
            self.astronomical_evolution_on = True
        
sphere = Sphere()

def mouse(event):
    global sphere
    star = vpython.scene.mouse.pick
    if star in sphere.vplanets:
        sphere.set_pole(sphere.vplanets.index(star), "surface")
    elif star in sphere.vqubits:
        sphere.set_pole(sphere.vqubits.index(star), "interior")
    elif star in sphere.veigs:
        i = sphere.veigs.index(star)
        if i == 0:
            sphere.collapse(0, -1)
        elif i == 1:
            sphere.collapse(0, 1)
        elif i == 2:
            sphere.collapse(1, -1)
        elif i == 3:
            sphere.collapse(1, 1)
        elif i == 4:
            sphere.collapse(2, -1)
        elif i == 5:
            sphere.collapse(2, 1)
    
vpython.scene.bind('click', mouse)

def keyboard(event):
    global sphere
    key = event.key
    if key == "a":
        sphere.sphere_step("latitude", -1)
    elif key == "d":
        sphere.sphere_step("latitude", 1)
    elif key == "w":
        sphere.sphere_step("longitude", 1)
    elif key == "s":
        sphere.sphere_step("longitude", -1)
    elif key == "z":
        sphere.sphere_step("step", -1)
    elif key == "x":
        sphere.sphere_step("step", 1)
    elif key == "f":
        sphere.mobius_step("A", -1)
    elif key == "h":
        sphere.mobius_step("A", 1)
    elif key == "t":
        sphere.mobius_step("N", 1)
    elif key == "g":
        sphere.mobius_step("N", -1)
    elif key == "v":
        sphere.mobius_step("K", -1)
    elif key == "b":
        sphere.mobius_step("K", 1)
    elif key == "j":
        sphere.qubit_step(0, -1)
    elif key == "l":
        sphere.qubit_step(0, 1)
    elif key == "i":
        sphere.qubit_step(1, 1)
    elif key == "k":
        sphere.qubit_step(1, -1)
    elif key == "n":
        sphere.qubit_step(2, -1)
    elif key == "m":
        sphere.qubit_step(2, 1)
    elif key == "e":
        sphere.toggle_hamiltonian()
    elif key == "r":
        sphere.toggle_hamiltonian_sign()
    elif key == "y":
        sphere.random_state()
    elif key == "u":
        sphere.random_hamiltonian()
    elif key == "o":
        sphere.reorient()
    elif key == "p":
        sphere.refresh()
    elif key == "q":
        sphere.toggle_astronomy()
    elif key == "c":
        sphere.toggle_astronomical_time()

vpython.scene.bind('keydown', keyboard)

while True:
    sphere.revolve()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>